In [1]:
## Imports
from ast import literal_eval
import pandas as pd
import hypernetx as hnx

In [2]:
df = pd.read_csv('eventcells_v2.csv',converters={"cellproperties":literal_eval})
dfp = pd.read_csv('eventprops_v2.csv',converters={"labels":literal_eval, "properties":literal_eval})

Preprocessing to aggregate and remove duplicates

In [3]:
cell_props = list(df.cellproperties.apply(lambda x: frozenset(x.keys())).unique().item())

In [4]:
for prop in cell_props:
    df[prop] = df.cellproperties.apply(lambda x: x[prop])

In [5]:
df = df.groupby(['0','1'], as_index=False)[['0','1',*cell_props]].agg(
    {"nodes":"first",
     "id":lambda x: x,
     "properties": lambda x: {"id": [i["id"] for i in x]} if len(x) > 1 else x,
     "reltype": lambda x: x
    })

In [6]:
df["cellproperties"] = df[cell_props].to_dict(orient='records')

In [7]:
df = df[['0','1','cellproperties']]

In [8]:
df

,0,1,cellproperties
0,17578,63419,"{'id': 138547, 'properties': {'id': '818092'},..."
1,17579,21286,"{'id': 138577, 'properties': {'id': '710506'},..."
2,17579,28275,"{'id': 137987, 'properties': {'id': '710508'},..."
3,17579,49109,"{'id': 138049, 'properties': {'id': '710507'},..."
4,17579,62035,"{'id': 15415, 'properties': {'id': '710505'}, ..."
...,...,...,...
5445,20470,27560,"{'id': 124495, 'properties': {'id': '540841'},..."
5446,20470,49148,"{'id': 124988, 'properties': {'id': '540840'},..."
5447,20470,61182,"{'id': 124963, 'properties': {'id': '540843'},..."
5448,20471,8231,"{'id': 204552, 'properties': {'id': '409016'},..."


In [9]:
dfp

,level,id,labels,properties
0,0,17578,[ConflictAttack],{'vid': 'http://www.isi.edu/gaia/events/b7ad2e...
1,1,63419,"[LocPositionRegion, Gpe]",{'vid': 'http://www.isi.edu/gaia/entities/0562...
2,0,17579,[ConflictAttack],{'vid': 'http://www.isi.edu/gaia/events/ec4031...
3,1,49109,[GpeUrbanareaCity],{'vid': 'http://www.isi.edu/gaia/entities/90f1...
4,0,17579,[ConflictAttack],{'vid': 'http://www.isi.edu/gaia/events/ec4031...
...,...,...,...,...
10991,1,61182,[GpeUrbanareaVillage],{'vid': 'http://www.isi.edu/gaia/entities/d5ed...
10992,0,20471,[ConflictAttack],{'vid': 'http://www.isi.edu/gaia/events/07e9cf...
10993,1,8231,[Per],{'vid': 'http://www.isi.edu/gaia/entities/f92f...
10994,0,20472,[ConflictAttack],{'vid': 'http://www.isi.edu/gaia/events/380b8f...


In [10]:
dfp = dfp.drop_duplicates(subset=['level','id'])

In [11]:
dfp

,level,id,labels,properties
0,0,17578,[ConflictAttack],{'vid': 'http://www.isi.edu/gaia/events/b7ad2e...
1,1,63419,"[LocPositionRegion, Gpe]",{'vid': 'http://www.isi.edu/gaia/entities/0562...
2,0,17579,[ConflictAttack],{'vid': 'http://www.isi.edu/gaia/events/ec4031...
3,1,49109,[GpeUrbanareaCity],{'vid': 'http://www.isi.edu/gaia/entities/90f1...
5,1,21286,[GpeCountryCountry],{'vid': 'http://www.isi.edu/gaia/entities/32aa...
...,...,...,...,...
10991,1,61182,[GpeUrbanareaVillage],{'vid': 'http://www.isi.edu/gaia/entities/d5ed...
10992,0,20471,[ConflictAttack],{'vid': 'http://www.isi.edu/gaia/events/07e9cf...
10993,1,8231,[Per],{'vid': 'http://www.isi.edu/gaia/entities/f92f...
10994,0,20472,[ConflictAttack],{'vid': 'http://www.isi.edu/gaia/events/380b8f...


In [12]:
H = hnx.Hypergraph(setsystem=df,properties=dfp,cell_properties="cellproperties")

In [13]:
H.edges.properties

labels  \
level id                                            
0     17578                      [ConflictAttack]   
      17579                      [ConflictAttack]   
      17580                      [ConflictAttack]   
      17581                      [ConflictAttack]   
      17582                      [ConflictAttack]   
...                                           ...   
1     404456  [OrgGovernmentLawenforcementagency]   
      404459  [OrgGovernmentLawenforcementagency]   
      404460  [OrgGovernmentLawenforcementagency]   
      404464  [OrgGovernmentLawenforcementagency]   
      404470  [OrgGovernmentLawenforcementagency]   

                                                     properties  
level id                                                         
0     17578   {'vid': 'http://www.isi.edu/gaia/events/b7ad2e...  
      17579   {'vid': 'http://www.isi.edu/gaia/events/ec4031...  
      17580   {'vid': 'http://www.isi.edu/gaia/events/ba0978...  
      17581   {'vid': 'http://www.isi.edu/gaia/events/336721...  
      17582   {'vid': 'http://www.isi.edu/gaia/events/97dc09...  
...                                                         ...  
1     404456  {'vid': 'http://www.isi.edu/gaia/entities/ebbb...  
      404459  {'vid': 'http://www.isi.edu/gaia/entities/58c4...  
      404460  {'vid': 'http://www.isi.edu/gaia/entities/b6ac...  
      404464  {'vid': 'http://www.isi.edu/gaia/entities/803c...  
      404470  {'vid': 'http://www.isi.edu/gaia/entities/910f...  

[6419 rows x 2 columns]

In [14]:
H.edges[17579].labels

['ConflictAttack']

In [15]:
H.edges[17579].properties

{'vid': 'http://www.isi.edu/gaia/events/ec403112-c356-4ff4-8b74-2e9aa8fa6c16-cluster-ss0HI8kE1K',
 'ldc_start_year': ['2016'],
 'event_type': 'ConflictAttack',
 'rdf_base_type': ['Event'],
 'ldc_end_month': ['04'],
 'ldc_start_month': ['04'],
 'ldc_end_year': ['2016'],
 'ldc_start_day': ['16'],
 'ldc_end_day': ['16']}

In [16]:
H.edges.set_property(17579,'test_prop','test_val')

In [17]:
H.edges[17579].properties

{'vid': 'http://www.isi.edu/gaia/events/ec403112-c356-4ff4-8b74-2e9aa8fa6c16-cluster-ss0HI8kE1K',
 'ldc_start_year': ['2016'],
 'event_type': 'ConflictAttack',
 'rdf_base_type': ['Event'],
 'ldc_end_month': ['04'],
 'ldc_start_month': ['04'],
 'ldc_end_year': ['2016'],
 'ldc_start_day': ['16'],
 'ldc_end_day': ['16'],
 'test_prop': 'test_val'}

In [18]:
H.edges[17579]

AttrList([21286, 28275, 49109, 62035, 63978])

In [19]:
H.edges.cell_properties.loc[(17579,21286)].item()

{'id': 138577,
 'properties': {'id': '710506'},
 'nodes': (17579, 21286),
 'reltype': 'conflict_attack_place'}

In [20]:
H.edges.set_cell_property(17579, 21286, "test_cell_prop", "test_cell_val")

In [21]:
H.edges.cell_properties.loc[(17579,21286)].item()

{'id': 138577,
 'properties': {'id': '710506'},
 'nodes': (17579, 21286),
 'reltype': 'conflict_attack_place',
 'test_cell_prop': 'test_cell_val'}

In [22]:
%%time
components = sorted(list(H.connected_component_subgraphs()), key=lambda x: x.number_of_nodes(), reverse=True)

CPU times: user 18.3 s, sys: 69.7 ms, total: 18.4 s
Wall time: 18.4 s


In [23]:
components[0].edges.properties

labels  \
level id                                            
0     17578                      [ConflictAttack]   
      17579                      [ConflictAttack]   
      17582                      [ConflictAttack]   
      17583                      [ConflictAttack]   
      17584                      [ConflictAttack]   
...                                           ...   
1     404306  [OrgGovernmentLawenforcementagency]   
      404316  [OrgGovernmentLawenforcementagency]   
      404392  [OrgGovernmentLawenforcementagency]   
      404426  [OrgGovernmentLawenforcementagency]   
      404460  [OrgGovernmentLawenforcementagency]   

                                                     properties  
level id                                                         
0     17578   {'vid': 'http://www.isi.edu/gaia/events/b7ad2e...  
      17579   {'vid': 'http://www.isi.edu/gaia/events/ec4031...  
      17582   {'vid': 'http://www.isi.edu/gaia/events/97dc09...  
      17583   {'vid': 'http://www.isi.edu/gaia/events/faa333...  
      17584   {'vid': 'http://www.isi.edu/gaia/events/d2ddf8...  
...                                                         ...  
1     404306  {'vid': 'http://www.isi.edu/gaia/entities/708c...  
      404316  {'vid': 'http://www.isi.edu/gaia/entities/3b17...  
      404392  {'vid': 'http://www.isi.edu/gaia/entities/2e87...  
      404426  {'vid': 'http://www.isi.edu/gaia/entities/5a80...  
      404460  {'vid': 'http://www.isi.edu/gaia/entities/b6ac...  

[2543 rows x 2 columns]

In [24]:
components[0].edges.cell_properties

cellproperties
0     1                                                       
17578 63419  {'id': 138547, 'properties': {'id': '818092'},...
17579 21286  {'id': 138577, 'properties': {'id': '710506'},...
      28275  {'id': 137987, 'properties': {'id': '710508'},...
      49109  {'id': 138049, 'properties': {'id': '710507'},...
      62035  {'id': 15415, 'properties': {'id': '710505'}, ...
...                                                        ...
20468 21070  {'id': 137147, 'properties': {'id': '567556'},...
      64424  {'id': 17895, 'properties': {'id': '567557'}, ...
20469 14711  {'id': 124461, 'properties': {'id': '848858'},...
      40293  {'id': 125697, 'properties': {'id': '848859'},...
      49173  {'id': 137123, 'properties': {'id': '848860'},...

[2943 rows x 1 columns]

In [26]:
%%time
hnx.dist_stats(H)

CPU times: user 88.8 ms, sys: 5.27 ms, total: 94 ms
Wall time: 92.3 ms


{'nrows': 3524,
 'ncols': 2895,
 'aspect ratio': 1.2172711571675303,
 'ncells': 5450,
 'density': 0.0005342100258969337,
 'node degree list': [211,
  103,
  94,
  57,
  55,
  47,
  47,
  43,
  32,
  30,
  30,
  29,
  28,
  24,
  22,
  21,
  18,
  18,
  18,
  17,
  16,
  15,
  15,
  14,
  14,
  14,
  12,
  11,
  11,
  11,
  11,
  11,
  10,
  10,
  9,
  9,
  9,
  9,
  9,
  8,
  8,
  8,
  8,
  8,
  8,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3